In [7]:
import pandas as pd
df=  pd.read_json("data/training_data.json")

scal_data = {}

# normalize columns
for column in ['dist','angle','yellow_distance']:
    scal_data[column] = [min(df[column]),max(df[column])]
    df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())



df

,img,lidar,dist,angle,yellow_distance
0,"[[[58, 72, 109], [58, 72, 109], [58, 72, 109],...","[0.6026701927185051, 0.610209167003631, 0.6181...",0.118589,0.000000,0.591787
1,"[[[67, 80, 121], [67, 80, 121], [66, 80, 119],...","[0.386225640773773, 0.388485670089721, 0.39086...",0.075204,0.000000,0.604903
2,"[[[73, 89, 131], [73, 89, 131], [72, 89, 129],...","[0.291639506816864, 0.291985154151916, 0.29238...",0.056245,0.000000,0.598141
3,"[[[87, 102, 147], [86, 102, 144], [85, 101, 14...","[0.25371536612510603, 0.252918452024459, 0.252...",0.046990,0.165354,0.601514
4,"[[[99, 114, 154], [99, 114, 156], [100, 115, 1...","[0.269550085067749, 0.26730242371559104, 0.265...",0.042236,0.385827,0.599741
...,...,...,...,...,...
2569,"[[[57, 67, 98], [57, 67, 97], [57, 67, 97], [5...","[0.12894400954246502, 0.12786881625652302, 0.1...",0.019050,0.385827,0.607161
2570,"[[[51, 60, 89], [51, 60, 88], [50, 59, 87], [5...","[0.18310509622097002, 0.18000471591949402, 0.1...",0.019024,0.614173,0.606480
2571,"[[[48, 61, 99], [48, 61, 97], [48, 61, 97], [4...","[0.49001839756965604, 0.469083726406097, 0.449...",0.023789,0.834646,0.607161
2572,"[[[49, 64, 99], [50, 63, 99], [50, 63, 99], [4...","[0.7098647356033321, 0.7091904878616331, 0.708...",0.032938,1.000000,0.607161


In [9]:
def normalize(val,column):
    return (val - scal_data[column][0]) / (scal_data[column][1] - scal_data[column][0])

def denormalize(val, column):
    return val * (scal_data[column][1] - scal_data[column][0]) + scal_data[column][0]


print(normalize(0.4, "dist"))
denormalize(0.0779651616857603, "dist")




0.0779651616857603


0.4000000000000001

In [14]:
print(normalize(100, "yellow_distance"))
print(denormalize(0.1822735148427958, "yellow_distance"))

0.1822735148427958
100.0


In [19]:
from sklearn.model_selection import train_test_split


# train test split 
imgs,lidars, distances, angles, yellow_distances = [],[],[],[], []

for _,row in df.iterrows():

    img,lidar = preprocessing(row['img'],row['lidar'])

    imgs.append(img)
    lidars.append(lidar)
    distances.append(row['dist'])
    angles.append(row['angle'])
    yellow_distances.append(row['yellow_distance'])


imgs = np.array(imgs, dtype=np.float32)
lidars = np.array(lidars, dtype=np.float32)
distances = np.array(distances, dtype=np.float32)
angles = np.array(angles, dtype=np.float32)
yellow_distances = np.array(yellow_distances, dtype=np.float32)

img_train, img_val, lidar_train, lidar_val, dist_train, dist_val, angle_train, angle_val, yellow_dist_train, yellow_dist_val = train_test_split(
    imgs, lidars, distances, angles, yellow_distances, test_size=0.2, random_state=42
)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

model = build_model()

# Callback
checkpoint = ModelCheckpoint("best_model.h5", monitor="val_loss", save_best_only=True)

# Treinamento
model.fit(
    [img_train, lidar_train],
    [dist_train, angle_train, yellow_dist_train],
    validation_data=([img_val, lidar_val], [dist_val, angle_val, yellow_dist_val]),
    epochs=20,
    batch_size=32,
    callbacks=[checkpoint]
)

In [37]:


def predict(img,lidar,single=True):
    img,lidar = preprocessing(img,lidar,single)
    distance_pred, angle_pred = model.predict([img, lidar])

def test():

    img,lidar,dist,angle,yellow_distance = df.iloc[11]

    img,lidar =  preprocessing(img,lidar, single=True)


    print(f"actual: {dist} {angle} {yellow_distance}")

    pdist,pangle,pyellow_distance = model.predict([img,lidar])
    print(f"predicted: {pdist} {pangle} {pyellow_distance}")

test()



actual: 0.054933447499299 0.9448818897637795 0.602245223516168
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
predicted: [[0.05001383]] [[0.93775195]] [[0.58166116]]


In [5]:
from controllers.find_ball.components.cnn.model import CNN

img,lidar,dist,angle,yellow_distance = df.iloc[2]

model = CNN()
pdist,pangle,pyellow_distance = model.predict(img,lidar)

print(f"actual: {dist} {angle} {yellow_distance}")
print(f"predicted: {pdist} {pangle} {pyellow_distance}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
actual: 0.056245129117985494 0.0 0.5981411379569618
predicted: [[0.04191741]] [[0.00406026]] [[0.5744534]]
